In [1]:
import librosa
from sklearn.model_selection import train_test_split
import numpy as np
#TODO
#Ver si vamos a usar Drive para los datasets
#from google.colab import drive
import os
import json
import tensorflow.keras as keras
import pickle #Para exportar el modelo

In [26]:
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#TODO
#Definir el path de los datos
data_path = "some_path/jsondataaudio.json"
epochs = 40 #Cantidad de ciclos completos de entrenamiento de todos los datos 
batch_size = 16 #Cantidad de datos utilizados en las iteraciones de entrenamientos
num_keywords = 2 #Numero de outputs, categorias
neuronas_por_capa = 128

In [4]:
def load_dataset(data_path):
  with open(data_path, "r") as fp:
    data = json.load(fp)

  #Transformamos las filas de mfcc y labels que estaban como listas a numpay arrays
  X = np.array(data["mfcc"])
  y = np.array(data["labels"])

  return X, y

In [5]:
def get_data_splits(data_path, test_size = 0.2, test_validation_size = 0.1):
  #cargamos el dataset
  X, y = load_dataset(data_path)

  #creamos los train/validation/test splits
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)
  X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = test_validation_size )

  return X_train, X_validation, X_test, y_train, y_validation, y_test

In [6]:
def predict(X, y, model):
  X = X[np.newaxis, ...]
  prediccion = model.predict(X)
  prediccion = np.argmax(prediccion, axis=1)
  print(f"Esperado: {y} Prediccion: {prediccion}")


In [7]:
def build_model(input_shape):
  #RNN
  #build network
  model = keras.Sequential()

  #lstm layer 1
  model.add(keras.layers.LSTM(neuronas_por_capa, input_shape = input_shape, return_sequences = True))

  #lstm layer 2 recibe la secuencia de layer 1, pero ahora devuelve un vector
  model.add(keras.layers.LSTM(neuronas_por_capa))

  #dense layer
  model.add(keras.layers.Dense(neuronas_por_capa, activation = "relu"))

  #para reducir overfitting
  model.add(keras.layers.Dropout(0.3))

  #softmax classifier
  model.add(keras.layers.Dense(num_keywords, activation = "softmax"))
  #compile the model

  return model

Cargar datos de entrenamiento/validacion/testeo

In [8]:
X_train, X_validation, X_test, y_train, y_validation, y_test = get_data_splits(data_path) 

Modelo

In [9]:
input_shape = (X_train.shape[1], X_train.shape[2]) #Es la forma de input que la RNN espera recibir
model = build_model(input_shape)

In [14]:
optimizador = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = optimizador,
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"]) #metrics despues nos ayudara a visualizar la precision con model.fit

Entrenar

In [16]:
model.fit(X_train,
          y_train,
          epochs = epochs,
          batch_size = batch_size,
          validation_data = (X_validation, y_validation)
          )          #validation_data = (X_validation, y_validation)

Epoch 1/40
450/450 [==============================] - 116s 259ms/step - loss: 1.7809 - accuracy: 0.3715 - val_loss: 1.5155 - val_accuracy: 0.4493
Epoch 2/40
450/450 [==============================] - 110s 243ms/step - loss: 1.4042 - accuracy: 0.4980 - val_loss: 1.3101 - val_accuracy: 0.5344
Epoch 3/40
450/450 [==============================] - 108s 241ms/step - loss: 1.2268 - accuracy: 0.5692 - val_loss: 1.1918 - val_accuracy: 0.5557
Epoch 4/40
450/450 [==============================] - 113s 251ms/step - loss: 1.1264 - accuracy: 0.6051 - val_loss: 1.1384 - val_accuracy: 0.6045
Epoch 5/40
450/450 [==============================] - 109s 242ms/step - loss: 1.0249 - accuracy: 0.6433 - val_loss: 1.0412 - val_accuracy: 0.6108
Epoch 6/40
450/450 [==============================] - 109s 243ms/step - loss: 0.9454 - accuracy: 0.6760 - val_loss: 1.0102 - val_accuracy: 0.6533
Epoch 7/40
450/450 [==============================] - 111s 247ms/step - loss: 0.8759 - accuracy: 0.7027 - val_loss: 0.9872 -

Guardar modelo para usarlo luego sin tener que volver a entrenarlo

In [30]:
#guardar el modelo
with open('/content/drive/My Drive/Machine Learning TP/Modelos/Audio/Genero_model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)


#model.save(save_path_model)

INFO:tensorflow:Assets written to: ram://0b7d49cf-4708-482b-b99a-3f7ef8bde11c/assets


INFO:tensorflow:Assets written to: ram://0b7d49cf-4708-482b-b99a-3f7ef8bde11c/assets
